### Схема тетрадки:
* [Импорт](#first-bullet)
* [Префильтрация](#second-bullet)
* [Модель 1 уровня: рекомендательная](#third-bullet)
* [Модель 2 уровня: ранжирующая](#forth-bullet)
* [Оценка результатов](#fith-bullet)
* [Сохранение результатов](#sixth-bullet)
---

***

***
# Блок импорта <a class="anchor" id="first-bullet"></a>
* импорт библиотек
* загрузка констант из файла `settings.yaml`

In [1]:
from src.utils import *
globals().update(load_settings(True))

Loaded following classes:
['Dataset', 'MainRecommender']

Loaded following functions:
['load_settings', 'pprint', 'precision_at_k']

Loaded following constants:
ITEM_COL             = item_id              | <class 'str'>
USER_COL             = user_id              | <class 'str'>
ACTUAL_COL           = actual               | <class 'str'>
TAKE_N_POPULAR       =                    5 | <class 'int'>
N_PREDICT            =                   50 | <class 'int'>
TOPK_PRECISION       =                    5 | <class 'int'>
VAL_MATCHER_WEEKS    =                    6 | <class 'int'>
VAL_RANKER_WEEKS     =                    3 | <class 'int'>
CATBOOST_RANKER      = YetiRankPairwise     | <class 'str'>
CATBOOST_ITERATIONS  =                 2000 | <class 'int'>


***
# Блок предоброботки данных <a class="anchor" id="second-bullet"></a>
* загрузка датасетов
* первичная трансформация датасетов
* добавление новых фич
* конвертация текстовых фичей в цифровые для модели второго уровня
* разделение и подготовка датасетов для формирования рекомендаций и последующего ранжирования

In [2]:
data = Dataset()
data.data_prefilter()
data.data_split()

train
test


***
# Блок формирования рекомендаций <a class="anchor" id="third-bullet"></a>
* train_test_split
* получение рекомендаций по собственным покупкам (с добавлением популярных товаров)

In [3]:
recommender = MainRecommender(data.data_train_lvl_1)
recommender.preprocessing(data, t='train')
recommender.preprocessing(data, t='test')

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/48263 [00:00<?, ?it/s]

In [4]:
recommender.recommend_1lvl(data.result_lvl_1).head(1)

,user_id,actual,own_recs,own_recs_score
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1049998, 1087895, 949257, 1032924, 6904485, 1...",0.132743


***
# Блок ранжирования полученных рекомендаций <a class="anchor" id="forth-bullet"></a>
* форматируем датасеты под формат (class Pool) CatBoost
* обучаем CatBoostRanker с функцией потерь YetiRankPairwise
* получаем предсказанные вероятности для ранжирования
* ранжируем рекомендации от модели первого уровня

In [5]:
recommender.ranker_fit()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [6]:
recommender.ranker_predict(recommender.X_train).head()

,user_id,item_id,manufacturer,brand,commodity_type,household_size_desc,kid_category_desc,age,income,has_kids,adults_num,Homeowner,Probable Owner,Probable Renter,Renter,Unknown,A,B,U,predict
0,1364,1014831,869,0,84,1,0,75.0,112.0,0.0,1.0,1,0,0,0,0,0,1,0,0.610433
1,1364,7169090,2110,0,9,1,0,75.0,112.0,0.0,1.0,1,0,0,0,0,0,1,0,0.400648
2,1364,1101378,2224,0,63,1,0,75.0,112.0,0.0,1.0,1,0,0,0,0,0,1,0,0.480235
3,1364,1090252,2224,0,63,1,0,75.0,112.0,0.0,1.0,1,0,0,0,0,0,1,0,0.480235
4,1364,7152319,2627,0,201,1,0,75.0,112.0,0.0,1.0,1,0,0,0,0,0,1,0,0.332908


***
# Блок оценки полученных результатов <a class="anchor" id="fith-bullet"></a>

In [7]:
recommender.evaluate_2models()

precision@5 of 2lvl-model is 0.0968586387434555


***
# Блок сохранения результата <a class="anchor" id="sixth-bullet"></a>
* повторяем все вышеописанные действия для "боевой" базы
* сохраняем результат в .xml

In [14]:
recommender = MainRecommender(data.data_train_lvl_1_real)
recommender.preprocessing(data, t='train', training=False)
recommender.preprocessing(data, t='test', training=False)
recommender.ranker_fit(training=False)
recommender.evaluate_2models(training=False)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


IndexError: Out of bounds on buffer access (axis 0)

***